<a href="https://colab.research.google.com/github/inna030/ETL/blob/main/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install awscli

In [ ]:
pip install pipenv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''import os
!export AWS_SHARED_CREDENTIALS_FILE=/content/drive/My\ Drive/config/awscli.ini
path = "/content/drive/My Drive/config/awscli.ini"
os.environ['AWS_SHARED_CREDENTIALS_FILE'] = path
print(os.environ['AWS_SHARED_CREDENTIALS_FILE'])'''

'import os\n!export AWS_SHARED_CREDENTIALS_FILE=/content/drive/My\\ Drive/config/awscli.ini\npath = "/content/drive/My Drive/config/awscli.ini"\nos.environ[\'AWS_SHARED_CREDENTIALS_FILE\'] = path\nprint(os.environ[\'AWS_SHARED_CREDENTIALS_FILE\'])'

In [ ]:
! pip install boto3

In [ ]:
import boto3

BUCKET_NAME = 'deutsche-boerse-xetra-pds' # replace with your bucket name

# enter authentication credentials
s3 = boto3.resource('s3', aws_access_key_id = '', 
                          aws_secret_access_key= '')

In [ ]:
!aws s3 ls deutsche-boerse-xetra-pds/2021-08-05/ --no-sign-request

2021-08-06 00:30:04        136 2021-08-05_BINS_XETR00.csv
2021-08-06 00:30:05        136 2021-08-05_BINS_XETR01.csv
2021-08-06 00:30:05        136 2021-08-05_BINS_XETR02.csv
2021-08-06 00:30:05        136 2021-08-05_BINS_XETR03.csv
2021-08-06 00:30:05        136 2021-08-05_BINS_XETR04.csv
2021-08-06 00:30:05        136 2021-08-05_BINS_XETR05.csv
2021-08-06 00:30:06        136 2021-08-05_BINS_XETR06.csv
2021-08-06 00:30:06    1414574 2021-08-05_BINS_XETR07.csv
2021-08-06 00:30:06    1212092 2021-08-05_BINS_XETR08.csv
2021-08-06 00:30:07    1077607 2021-08-05_BINS_XETR09.csv
2021-08-06 00:30:07    1014529 2021-08-05_BINS_XETR10.csv
2021-08-06 00:30:07    1124025 2021-08-05_BINS_XETR11.csv
2021-08-06 00:30:07     934216 2021-08-05_BINS_XETR12.csv
2021-08-06 00:30:07    1248285 2021-08-05_BINS_XETR13.csv
2021-08-06 00:30:08    1275513 2021-08-05_BINS_XETR14.csv
2021-08-06 00:30:08    1171325 2021-08-05_BINS_XETR15.csv
2021-08-06 00:30:08        136 2021-08-05_BINS_XETR16.csv
2021-08-06 00:

In [ ]:
!pip install pandas

In [ ]:
from io import StringIO,BytesIO
from datetime import datetime,timedelta

In [ ]:
import pandas as pd

In [ ]:
def read_csv_to_df(bucket, key, decoding = 'utf-8', sep = ','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(decoding)
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=sep)
    return df

def write_df_to_s3(bucket, df, key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer, index=False)
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True

def write_df_to_s3_csv(bucket, df, key):
    out_buffer = StringIO()
    df.to_csv(out_buffer, index=False)
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True

def list_files_in_prefix(bucket, prefix):
    files = [obj.key for obj in bucket.objects.filter(Prefix=prefix)]
    return files

meta_key='aws-s3.csv'
bucket_name_trg='inna030'
s3 = boto3.resource('s3', aws_access_key_id = '', 
                          aws_secret_access_key= '')


Application Layer - not core

In [ ]:
def return_date_list(bucket, arg_date, src_format, meta_key):
    min_date = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)
    today = datetime.today().date()
    try:
        df_meta = read_csv_to_df(bucket, meta_key)
        dates = [(min_date + timedelta(days=x)) for x in range(0, (today-min_date).days + 1)]
        src_dates = set(pd.to_datetime(df_meta['source_date']).dt.date)
        dates_missing = set(dates[1:]) - src_dates
        if dates_missing:
            min_date = min(set(dates[1:]) - src_dates) - timedelta(days=1)
            return_dates = [date.strftime(src_format) for date in dates if date >= min_date]
            return_min_date = (min_date + timedelta(days=1)).strftime(src_format)
        else:
            return_dates = []
            return_min_date = datetime(2200, 1, 1).date()
    except bucket.session.client('s3').execptions.NoSuchKey:
        return_dates = [(min_date + timedelta(days=x)).strftime(src_format) for x in range(0, (today-min_date).days + 1)]
        return_min_date = arg_date
    return return_min_date, return_dates

def update_meta_file(bucket, meta_key, extract_date_list):
    df_new = pd.DataFrame(columns=['source_date', 'datetime_of_processing'])
    df_new['source_date'] = extract_date_list
    df_new['datetime_of_processing'] = datetime.today().strftime('%Y-%m-%d')
    df_old = read_csv_to_df(bucket, meta_key)
    df_all = pd.concat([df_old, df_new])
    write_df_to_s3_csv(bucket, df_all, meta_key)
    



Adapter Layer

In [ ]:
def read_csv_to_df(bucket,key,decoding='utf-8',sep=','):
  csv_obj=bucket.Object(key=key).get().get('Body').read().decode(decoding)
  data=StringIO(csv_obj)
  df=pd.read_csv(data,delimiter=sep)
  return df

def write_df_to_s3(bucket,df,key):
  out_buffer=BytesIO()
  df_all.to_parquet(out_buffer,index=False)
  bucket.put_object(Body=out_buffer.getvalue(),key=key)
  return True


def write_df_to_s3_csv(bucket,df,key):
  out_buffer=BytesIO()
  df_all.to_csv(out_buffer,index=False)
  bucket.put_object(Body=out_buffer.getvalue(),key=key)
  return True

def list_files_in_prefix(bucket,prefix):
  files=[i.key for i in bucket.objects.filter(Prefix=prefix)]
  return files



Application Layer

In [ ]:
def extract(bucket, date_list):
    files = [key for date in date_list for key in list_files_in_prefix(bucket, date)]
    df = pd.concat([read_csv_to_df(bucket, obj) for obj in files], ignore_index=True)
    return df

def transform_report1(df, columns, arg_date):
    df = df.loc[:, columns]
    df.dropna(inplace=True)
    df['opening_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
    df['closing_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')
    df = df.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))
    df['prev_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df['change_prev_closing_%'] = (df['closing_price_eur'] - df['prev_closing_price']) / df['prev_closing_price'] * 100
    df.drop(columns=['prev_closing_price'], inplace=True)
    df = df.round(decimals=2)
    df = df[df.Date >= arg_date]
    return df

def load(bucket, df, trg_key, trg_format, meta_key, extract_date_list):
    key = trg_key + datetime.today().strftime("%Y%m%d_%H%M%S") + trg_format
    write_df_to_s3(bucket, df, key)
    update_meta_file(bucket, meta_key, extract_date_list)
    return True

def etl_report1(src_bucket, trg_bucket, date_list, columns, arg_date, trg_key, trg_format, meta_key):
    df = extract(src_bucket, date_list)
    df = transform_report1(df, columns, arg_date)
    extract_date_list = [date for date in date_list if date >= arg_date]
    load(trg_bucket, df, trg_key, trg_format, meta_key, extract_date_list)
    return True
    

Application layer

In [ ]:
def return_date_list(bucket,arg_date,src_format):
  min_date=datetime.strptime(arg_date,src_format).date()-timedelta(days=1)
  today=datetime.today().date()
  return today

Main function entrypoint

In [ ]:
def main():
    # Parameters/Configurations
    # Later read config
    arg_date = '2021-08-09'
    src_format = '%Y-%m-%d'
    src_bucket = 'deutsche-boerse-xetra-pds'
    trg_bucket = 'inna030'
    columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
    trg_key = 'xetra_daily_report_'
    trg_format = '.parquet'
    meta_key = 'aws-s3.csv'
    
    # Init
    s3 = boto3.resource('s3', aws_access_key_id = '', 
                          aws_secret_access_key= '')
    # run application
    bucket_src = s3.Bucket(src_bucket)
    bucket_trg = s3.Bucket(trg_bucket)
    # run application
    date_list = return_date_list(bucket_trg, arg_date, src_format)
    etl_report1(bucket_src, bucket_trg, date_list, columns, arg_date, trg_key, trg_format, meta_key)



In [ ]:

def read_csv_to_df(bucket,key,decoding='utf-8',sep=','):
  print("Extracting key: {}".format(key))

In [ ]:
#Run
main()

In [ ]:
bucket=s3.Bucket(src_bucket)

In [ ]:
def csv_to_df(filename):
  csv_obj=bucket.Object(key=filename).get().get('Body').read().decode('utf-8')
  data=StringIO(csv_obj)
  df=pd.read_csv(data,delimiter=',')
  return df

In [ ]:
df_all

In [ ]:
arg_date='2021-05-07'

In [ ]:
csv_obj_columnname=bucket.Object(key=objects[0].key).get().get('Body').read().decode('utf-8')
data=StringIO(csv_obj_columnname)
df_init=pd.read_csv(data,delimiter=',')


In [ ]:
columns=['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']


,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume
0,AT0000A0E9W5,2021-03-15,08:00,22.120,22.120,22.120,22.120,1527
1,DE000A0DJ6J9,2021-03-15,08:00,53.850,53.850,53.500,53.500,508
2,DE000A0D6554,2021-03-15,08:00,22.240,22.240,22.180,22.180,5270
3,DE000A0D9PT0,2021-03-15,08:00,201.500,201.500,200.600,200.600,1744
4,DE000A0HN5C6,2021-03-15,08:00,38.950,39.060,38.890,39.050,28662
...,...,...,...,...,...,...,...,...
206907,DE000A0WMPJ6,2021-03-16,16:42,19.235,19.235,19.235,19.235,211
206908,DE0007164600,2021-03-16,16:43,103.020,103.020,103.020,103.020,20
206909,DE0007568578,2021-03-16,16:44,24.550,24.550,24.550,24.550,130
206910,DE0007664005,2021-03-16,16:44,266.600,266.600,266.600,266.600,500


In [ ]:
! pip install pyarrow

Write to S3

In [ ]:

bucket_target=s3.Bucket(trg_bucket)


In [ ]:
trg_bucket='inna030'
s3 = boto3.resource('s3', aws_access_key_id = '', 
                          aws_secret_access_key= '')
bucket_trg=s3.Bucket(trg_bucket)
for i in bucket_tar.objects.all():
  print(i.key)

Reading the uploaded file

In [ ]:
prq_obj=bucket_tar.Object(key=i.key).get().get('Body').read()
data=BytesIO(prq_obj)
df_report=pd.read_parquet(data)

In [ ]:
df_report

In [ ]:
%cd drive/My Drive/ETL

/content/drive/My Drive/ETL


In [ ]:
! git clone https://github.com/inna030/ETL.git

Cloning into 'ETL'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), done.


In [ ]:
! git checkout -b develop

fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


S3.py

In [ ]:
"""Connector and methods accessing S3"""
import os
import boto3
import logging

In [ ]:
class s3bucketconnector():
  def __init__(self,access_key:str,secret_key:str,endpoint_url:str,bucket:str):
    #Constructor for S3BucketConnector
    self._logger=logging.getLogger(__name__)
    self.endpoint_url=endpoint_url
    self.session=boto3.Session(aws_access_key_id=,aws_scret_access_key=)
    self._s3=self.session.resource(service_name='s3',endpoint_url=endpoint_url)
    self._bucket=self._s3.Bucket(bucket)
  
  def list_files_in_prefix(self,prefix:str):
    """
    Listing all files with a preix on the S3 bucket
    :param prefix on the S3 bucket that should be filetered with

    returns:
      files:list of all the file names containing the prefix in the key
    """

    files=[i.key for i in self._bucket.objects.filter(Prefix=prefix)]
    return files
  
  def read_csv_to_df(self):
    pass

  def write_df_to_s3(self):
    pass


meta_process.py

In [ ]:
'''Methods for processing the meta file'''

class MetaProcess():
  '''
  class for working with the meta file
  '''

  @staticmethod
  def update_meta_file():
    pass

  @staticmethod
  def return_date_file():
    pass  


Constants.py

In [ ]:
'''
File to store constants
'''
from enum import Enum

class S3FileTypes(Enum):
  '''
  supported file types for S3BucketConnector
  '''
  CSV='csv'
  PARQUET='parquet'

class MetaProcessFormat(Enum):
  '''
  formation for MetaProcess Class
  '''
  META_DATE_FORMAT='%Y-%m-%d'
  META_PROCESS_DATE_FORMAT='%Y-%m-%d %H:%M:%S'
  META_SOURCE_DATE_COL='source_date'
  META_PROCESS_COL='datetime_of_processing'
  META_FILE_FORMAT='csv'

Custom_exceptions.py

In [ ]:
'''Custom Exceptions'''


Transformer.py

In [ ]:
"""Xetra ETL Component"""
from typing import NamedTuple
import logging
from xetra.common.s3 import S3BucketConnector

class XetraSourceConfig(NamedTuple):
    """
    Class for source configuration data

    src_first_extract_date: determines the date for extracting the source
    src_columns: source column names
    src_col_date: column name for date in source
    src_col_isin: column name for isin in source
    src_col_time: column name for time in source
    src_col_start_price: column name for starting price in source
    src_col_min_price: column name for minimum price in source
    src_col_max_price: column name for maximum price in source
    src_col_traded_vol: column name for traded volumne in source
    """
    self._logger=logging.getLogger(__name__)
    src_first_extract_date: str
    src_columns: list
    src_col_date: str
    src_col_isin: str
    src_col_time: str
    src_col_start_price: str
    src_col_min_price: str
    src_col_max_price: str
    src_col_traded_vol: str


class XetraTargetConfig(NamedTuple):
    """
    Class for target configuration data

    trg_col_isin: column name for isin in target
    trg_col_date: column name for date in target
    trg_col_op_price: column name for opening price in target
    trg_col_clos_price: column name for closing price in target
    trg_col_min_price: column name for minimum price in target
    trg_col_max_price: column name for maximum price in target
    trg_col_dail_trad_vol: column name for daily traded volume in target
    trg_col_ch_prev_clos: column name for change to previous day's closing price in target
    trg_key: basic key of target file
    trg_key_date_format: date format of target file key
    trg_format: file format of the target file
    """
    trg_col_isin: str
    trg_col_date: str
    trg_col_op_price: str
    trg_col_clos_price: str
    trg_col_min_price: str
    trg_col_max_price: str
    trg_col_dail_trad_vol: str
    trg_col_ch_prev_clos: str
    trg_key: str
    trg_key_date_format: str
    trg_format: str

class XetraETL():
    """
    Reads the Xetra data, transforms and writes the transformed to target
    """

    def __init__(self, s3_bucket_src: S3BucketConnector,
                 s3_bucket_trg: S3BucketConnector, meta_key: str,
                 src_args: XetraSourceConfig, trg_args: XetraTargetConfig):
        """
        Constructor for XetraTransformer

        :param s3_bucket_src: connection to source S3 bucket
        :param s3_bucket_trg: connection to target S3 bucket
        :param meta_key: used as self.meta_key -> key of meta file
        :param src_args: NamedTouple class with source configuration data
        :param trg_args: NamedTouple class with target configuration data
        """
        self.s3_bucket_src = s3_bucket_src
        self.s3_bucket_trg = s3_bucket_trg
        self.meta_key = meta_key
        self.src_args = src_args
        self.trg_args = trg_args
        self.extract_date =
        self.extract_date_list =
        self.meta_update_list = 
    
    def extract(self):
        pass

    def transform_report1(self):
        pass
    
    def load(self):
        pass
    
    def etl_report1(self):
        pass


Run.py

In [ ]:
import logging
import logging.config

def main():
  """entry point to run the xetra ETL job"""
  #Parsing YAML file
  config_path=''
  config=yaml.safe_load(open(config_path))
  #configure logging
  log_config=config['logging']
  logging.config.dictConfig(log_config)
  logger=logging.getLogger(__name__)
  logger.info("This is a test.")
if __name__=='__main__':
  main()

Config.yml

In [ ]:
#Logging configuration

logging:
  version:1
  formatters:
    xetra:
      format: "Xetra Transformer-%(asctime)s-%(levelname)s-%(message)s"
  handlers:
    console:
      class: logging.StreamHandler
      formatter: xetra
      level: DEBUG
  root:
    level: DEBUG
    handlers: [console]




Bash

In [ ]:
from xetra.common.constants import S3FileTypes
exit()
dir
cd tests
import sys
for p in sys.path:print(p)

from xetra.common.constants import S3FileTypes

Test_s3.py